In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_curve, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
test_df.info(50)

NameError: name 'test_df' is not defined

In [3]:
# Load dataset
train_df = pd.read_csv("merged_cleaned_train_flight_weather_data.csv")
test_df = pd.read_csv("merged_cleaned_test_flight_weather_data.csv")


In [5]:
# Convert to datetime objects
test_df['departure.scheduledTime'] = pd.to_datetime(test_df['departure.scheduledTime'], errors='coerce')
test_df['arrival.actualTime'] = pd.to_datetime(test_df['arrival.actualTime'], errors='coerce')


In [7]:
# Compute delay in minutes for test data
test_df['departure.delay_minutes'] = (test_df['arrival.actualTime'] - test_df['departure.scheduledTime']).dt.total_seconds() / 60

In [ ]:
# Handle missing values
train_df.fillna(train_df.mean(), inplace=True)
test_df.fillna(test_df.mean(), inplace=True)


In [9]:


# Create binary target: on-time vs delayed
train_df['delay_status'] = train_df['departure.delay_minutes'].apply(lambda x: 'on-time' if x == 0 else 'delayed')
test_df['delay_status'] = test_df['departure.delay_minutes'].apply(lambda x: 'on-time' if x == 0 else 'delayed')

# Create multi-class target: delay categories
train_df['delay_category'] = pd.cut(train_df['departure.delay_minutes'], bins=[-1, 0, 45, 175, float('inf')],
                                    labels=['No Delay', 'Short Delay', 'Moderate Delay', 'Long Delay'])
test_df['delay_category'] = pd.cut(test_df['departure.delay_minutes'], bins=[-1, 0, 45, 175, float('inf')],
                                   labels=['No Delay', 'Short Delay', 'Moderate Delay', 'Long Delay'])


In [13]:

# Convert time columns to datetime
train_df['departure.scheduledTime'] = pd.to_datetime(train_df['departure.scheduledTime'])
train_df['arrival.scheduledTime'] = pd.to_datetime(train_df['arrival.scheduledTime'])
test_df['departure.scheduledTime'] = pd.to_datetime(test_df['departure.scheduledTime'])
test_df['arrival.scheduledTime'] = pd.to_datetime(test_df['arrival.scheduledTime'])




In [15]:


# Features and labels for binary classification
X_train_binary = train_df.drop(columns=['departure.delay_minutes', 'delay_status', 'status_encoded'])
y_train_binary = train_df['delay_status']
X_test_binary = test_df.drop(columns=['departure.delay_minutes', 'delay_status', 'status_encoded'])
y_test_binary = test_df['delay_status']



In [17]:
# Features and labels for multi-class classification
X_train_multi = train_df.drop(columns=['departure.delay_minutes', 'delay_category', 'status_encoded'])
y_train_multi = train_df['delay_category']
X_test_multi = test_df.drop(columns=['departure.delay_minutes', 'delay_category', 'status_encoded'])
y_test_multi = test_df['delay_category']


In [19]:
from sklearn.preprocessing import LabelEncoder

# Example of binary label encoding
label_encoder = LabelEncoder()
X_train_binary['departure.day_of_week_Monday'] = label_encoder.fit_transform(X_train_binary['departure.day_of_week_Monday'])
X_test_binary['departure.day_of_week_Monday'] = label_encoder.transform(X_test_binary['departure.day_of_week_Monday'])


In [28]:
# Use sparse matrices
X_train_binary = pd.get_dummies(X_train_binary, drop_first=True, sparse=True)
X_test_binary = pd.get_dummies(X_test_binary, drop_first=True, sparse=True)


In [ ]:

# Encode categorical variables
#X_train_binary = pd.get_dummies(X_train_binary, drop_first=True)
#X_test_binary = pd.get_dummies(X_test_binary, drop_first=True)
X_train_multi = pd.get_dummies(X_train_multi, drop_first=True,sparse=True)
X_test_multi = pd.get_dummies(X_test_multi, drop_first=True,sparse=True)



In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train_binary_scaled = scaler.fit_transform(X_train_binary)
X_test_binary_scaled = scaler.transform(X_test_binary)
#X_train_multi_scaled = scaler.fit_transform(X_train_multi)
#X_test_multi_scaled = scaler.transform(X_test_multi)

In [66]:
print(train_df.info(50))
test_df.info(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51871 entries, 0 to 51870
Data columns (total 153 columns):
 #    Column                                        Dtype         
---   ------                                        -----         
 0    status                                        int64         
 1    departure.iataCode                            object        
 2    departure.icaoCode                            object        
 3    departure.terminal                            object        
 4    departure.scheduledTime                       datetime64[ns]
 5    departure.estimatedTime                       object        
 6    departure.actualTime                          object        
 7    departure.estimatedRunway                     object        
 8    departure.actualRunway                        object        
 9    arrival.scheduledTime                         datetime64[ns]
 10   arrival.estimatedTime                         object        
 11   airline.iataC

In [67]:
train_airline_columns = [col for col in train_df.columns if col.startswith('airline.name_')]
test_airline_columns = [col for col in test_df.columns if col.startswith('airline.name_')]


In [71]:
len(test_airline_columns)

73

In [75]:
# Ensure the list of airline columns is consistent with the test dataset
airline_columns = [col for col in test_df.columns if col.startswith('airline.name_')]

# Create a new column 'airline.name' by extracting the airline name from the boolean columns
def get_airline_name(row):
    for col in airline_columns:
        # Check if the column is in the dataframe and if the value is True
        if col in row and row[col]:
            return col.split('name_')[-1].replace('_', ' ')  # Get the airline name
    return 'Unknown'  # In case no airline column is True

# Apply the function to create the new column
test_df['airline.name'] = test_df.apply(get_airline_name, axis=1)


In [73]:
# Create a new column 'airline.name' by extracting the airline name from the boolean columns
def get_airline_name(row):
    for col in airline_columns:
        if row[col]:  # If the column is True
            return col.split('name_')[-1].replace('_', ' ')  # Get the airline name
    return 'Unknown'  # In case no airline column is True

#train_df['airline.name'] = train_df.apply(get_airline_name, axis=1)
test_df['airline.name'] = test_df.apply(get_airline_name, axis=1)


KeyError: 'airline.name_air hamburg'

In [44]:
train_df

,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.scheduledTime,departure.estimatedTime,departure.actualTime,departure.estimatedRunway,departure.actualRunway,arrival.scheduledTime,...,Precipitation (in) Total,Departure_DayOfWeek_Monday,Departure_DayOfWeek_Saturday,Departure_DayOfWeek_Sunday,Departure_DayOfWeek_Thursday,Departure_DayOfWeek_Tuesday,Departure_DayOfWeek_Wednesday,delay_status,delay_category,airline.name
0,0,lhe,opla,m,2023-07-20 20:50:00,2023-07-20 20:00:00,2023-07-20 20:15:00,2023-07-20 20:15:00,2023-07-20 20:15:00,2023-07-20 23:20:00,...,0.0,False,False,False,True,False,False,delayed,NaN,saudia
1,0,lhe,opla,Unknown,2023-07-18 15:05:00,2023-07-18 15:05:00,2023-07-18 15:05:00,2023-07-20 20:15:00,2023-07-18 15:05:00,2023-07-18 16:50:00,...,0.0,False,False,False,False,True,False,on-time,No Delay,flyjinnah
2,0,lhe,opla,Unknown,2023-07-23 09:50:00,2023-07-18 15:05:00,2023-07-23 09:50:00,2023-07-20 20:15:00,2023-07-23 09:50:00,2023-07-23 11:35:00,...,0.0,False,False,True,False,False,False,on-time,No Delay,flyjinnah
3,0,lhe,opla,m,2023-07-26 23:30:00,2023-07-26 23:30:00,2023-07-26 23:51:00,2023-07-26 23:51:00,2023-07-26 23:51:00,2023-07-27 01:30:00,...,0.0,False,False,False,False,False,True,delayed,Short Delay,pakistan international airlines
4,0,lhe,opla,m,2023-07-20 11:35:00,2023-07-20 17:15:00,2023-07-20 17:15:00,2023-07-26 23:51:00,2023-07-20 11:35:00,2023-07-20 14:00:00,...,0.0,False,False,False,True,False,False,delayed,Long Delay,serene air
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51866,0,lhe,opla,Unknown,2023-11-16 09:50:00,2023-11-16 09:50:00,2023-11-16 10:03:00,2023-11-16 10:03:00,2023-11-16 10:03:00,2023-11-16 11:45:00,...,0.0,False,False,False,True,False,False,delayed,Short Delay,flyjinnah
51867,0,lhe,opla,m,2023-11-16 11:40:00,2023-11-16 11:40:00,2023-11-16 11:48:00,2023-11-16 11:48:00,2023-11-16 11:48:00,2023-11-16 15:25:00,...,0.0,False,False,False,True,False,False,delayed,Short Delay,saudia
51868,0,lhe,opla,m,2023-11-27 10:50:00,2023-11-27 14:35:00,2023-11-27 14:35:00,2023-11-16 11:48:00,2023-11-27 10:50:00,2023-11-27 13:30:00,...,0.0,True,False,False,False,False,False,delayed,Long Delay,pakistan international airlines
51869,0,lhe,opla,m,2023-11-19 02:00:00,2023-11-19 11:00:00,2023-11-19 11:00:00,2023-11-16 11:48:00,2023-11-19 02:00:00,2023-11-19 10:30:00,...,0.0,False,False,True,False,False,False,delayed,Long Delay,pakistan international airlines


In [80]:
#train_df.drop(columns=airline_columns, inplace=True)

test_df.drop(columns=airline_columns, inplace=True)


In [114]:
print(train_df.info())
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51871 entries, 0 to 51870
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   status                           51871 non-null  int64         
 1   departure.iataCode               51871 non-null  object        
 2   departure.icaoCode               51871 non-null  object        
 3   departure.terminal               51871 non-null  object        
 4   departure.scheduledTime          51871 non-null  datetime64[ns]
 5   departure.estimatedTime          51871 non-null  object        
 6   departure.actualTime             51871 non-null  object        
 7   departure.estimatedRunway        51871 non-null  object        
 8   departure.actualRunway           51871 non-null  object        
 9   arrival.scheduledTime            51871 non-null  datetime64[ns]
 10  arrival.estimatedTime            51871 non-null  object   

In [119]:
# Check if 'airline.name' column exists in test_df, and add it if missing
if 'airline.name' not in test_df.columns:
    # Generate airline name column based on boolean columns, assuming 'airline.name_' columns are present
    airline_columns = [col for col in train_df.columns if col.startswith('airline.name_')]  # List of airline columns
    def get_airline_name(row):
        for col in airline_columns:
            if col in row and row[col]:  # Ensure column is present in the row and the value is True
                return col.split('name_')[-1].replace('_', ' ')  # Extract airline name
        return 'Unknown'  # Default value if no airline column is True
    
    test_df['airline.name'] = test_df.apply(get_airline_name, axis=1)

# Check if 'airline.name' is still missing (e.g., due to different boolean columns in test_df)
if 'airline.name' not in test_df.columns:
    test_df['airline.name'] = 'Unknown'  # Assign default value if still missing


In [125]:
print(test_df)
print(train_df)

       status departure.iataCode departure.icaoCode departure.scheduledTime  \
0           0                lhe               opla     2023-07-17 20:35:00   
1           0                lhe               opla     2023-07-27 08:00:00   
2           0                lhe               opla     2023-07-27 08:00:00   
3           2                lhe               opla     2023-07-28 16:45:00   
4           0                lhe               opla     2023-07-19 04:15:00   
...       ...                ...                ...                     ...   
14977       0                lhe               opla     2023-11-23 23:15:00   
14978       0                lhe               opla     2023-11-25 18:45:00   
14979       0                lhe               opla     2023-11-25 02:55:00   
14980       0                lhe               opla     2023-11-27 08:35:00   
14981       0                lhe               opla     2023-11-26 01:45:00   

      departure.estimatedRunway departure.actualRun

In [127]:
# Create a single column for the day of the week using numeric representation or categories
day_of_week_columns = [
    'Departure_DayOfWeek_Monday',
    'Departure_DayOfWeek_Saturday',
    'Departure_DayOfWeek_Sunday',
    'Departure_DayOfWeek_Thursday',
    'Departure_DayOfWeek_Tuesday',
    'Departure_DayOfWeek_Wednesday'
]

# Map each column to a numeric value for day of the week
day_map = {
    'Departure_DayOfWeek_Monday': 0,
    'Departure_DayOfWeek_Tuesday': 1,
    'Departure_DayOfWeek_Wednesday': 2,
    'Departure_DayOfWeek_Thursday': 3,
    'Departure_DayOfWeek_Friday': 4,
    'Departure_DayOfWeek_Saturday': 5,
    'Departure_DayOfWeek_Sunday': 6
}



In [129]:
# Create a new column for the day of the week by checking which boolean column is True
def get_day_of_week(row):
    for col in day_of_week_columns:
        if row[col]:
            return day_map[col]  # Return the corresponding day number
    return -1  # In case no day column is True

# Apply the function to both train and test dataframes
train_df['departure.day_of_week'] = train_df.apply(get_day_of_week, axis=1)
test_df['departure.day_of_week'] = test_df.apply(get_day_of_week, axis=1)


In [131]:

# Drop the individual day-of-week boolean columns
train_df.drop(columns=day_of_week_columns, inplace=True)
test_df.drop(columns=day_of_week_columns, inplace=True)

# Verify the result
print(train_df[['departure.day_of_week']].head())
print(test_df[['departure.day_of_week']].head())


   departure.day_of_week
0                      3
1                      1
2                      6
3                      2
4                      3
   departure.day_of_week
0                      0
1                      3
2                      3
3                     -1
4                      2


In [139]:
train_df.info()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51871 entries, 0 to 51870
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   status                           51871 non-null  int64         
 1   departure.iataCode               51871 non-null  object        
 2   departure.icaoCode               51871 non-null  object        
 3   departure.terminal               51871 non-null  object        
 4   departure.scheduledTime          51871 non-null  datetime64[ns]
 5   departure.estimatedTime          51871 non-null  object        
 6   departure.actualTime             51871 non-null  object        
 7   departure.estimatedRunway        51871 non-null  object        
 8   departure.actualRunway           51871 non-null  object        
 9   arrival.scheduledTime            51871 non-null  datetime64[ns]
 10  arrival.estimatedTime            51871 non-null  object   

In [157]:
train_df['departure.estimatedRunway']

0        2023-07-20 20:15:00
1        2023-07-20 20:15:00
2        2023-07-20 20:15:00
3        2023-07-26 23:51:00
4        2023-07-26 23:51:00
                ...         
51866    2023-11-16 10:03:00
51867    2023-11-16 11:48:00
51868    2023-11-16 11:48:00
51869    2023-11-16 11:48:00
51870    2023-11-23 06:47:00
Name: departure.estimatedRunway, Length: 51871, dtype: object

In [171]:
print(train_df['departure.day_of_week'].value_counts())
test_df['departure.day_of_week'].value_counts()


departure.day_of_week
 6    7612
-1    7523
 3    7491
 5    7446
 1    7335
 2    7260
 0    7204
Name: count, dtype: int64


departure.day_of_week
 3    2244
-1    2194
 1    2164
 6    2149
 0    2101
 5    2085
 2    2045
Name: count, dtype: int64

In [159]:
train_df.info()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51871 entries, 0 to 51870
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   status                           51871 non-null  int64         
 1   departure.iataCode               51871 non-null  object        
 2   departure.icaoCode               51871 non-null  object        
 3   departure.terminal               51871 non-null  object        
 4   departure.scheduledTime          51871 non-null  datetime64[ns]
 5   departure.estimatedTime          51871 non-null  object        
 6   departure.actualTime             51871 non-null  object        
 7   departure.estimatedRunway        51871 non-null  object        
 8   departure.actualRunway           51871 non-null  object        
 9   arrival.scheduledTime            51871 non-null  datetime64[ns]
 10  arrival.estimatedTime            51871 non-null  object   

In [163]:
train_df['departure.day_of_week_Monday']

0        False
1        False
2        False
3        False
4        False
         ...  
51866    False
51867    False
51868     True
51869    False
51870    False
Name: departure.day_of_week_Monday, Length: 51871, dtype: bool

In [165]:
# Define the mapping for each boolean column to the corresponding day number
day_map = {
    'departure.day_of_week_Monday': 0,
    'departure.day_of_week_Saturday': 5,
    'departure.day_of_week_Sunday': 6,
    'departure.day_of_week_Thursday': 3,
    'departure.day_of_week_Tuesday': 1,
    'departure.day_of_week_Wednesday': 2
}

# Define the list of boolean columns (days of the week)
day_of_week_columns = [
    'departure.day_of_week_Monday', 'departure.day_of_week_Saturday',
    'departure.day_of_week_Sunday', 'departure.day_of_week_Thursday',
    'departure.day_of_week_Tuesday', 'departure.day_of_week_Wednesday'
]



In [167]:
# Function to map boolean columns into a single day of the week number
def get_day_of_week(row):
    for col in day_of_week_columns:
        if row[col]:
            return day_map[col]  # Return the corresponding day number
    return -1  # In case no day column is True (shouldn't happen if the data is consistent)

# Apply the function to both train and test dataframes
train_df['departure.day_of_week'] = train_df.apply(get_day_of_week, axis=1)
test_df['departure.day_of_week'] = test_df.apply(get_day_of_week, axis=1)


In [169]:

# Drop the original boolean day-of-week columns
train_df.drop(columns=day_of_week_columns, inplace=True)
test_df.drop(columns=day_of_week_columns, inplace=True)

# Verify the result
print(train_df[['departure.day_of_week']].head())
print(test_df[['departure.day_of_week']].head())


   departure.day_of_week
0                      3
1                      1
2                      6
3                      2
4                      3
   departure.day_of_week
0                      0
1                      3
2                      3
3                     -1
4                      2


In [ ]:
# Verify the result
print(train_df[['departure.day_of_week']].head())
print(test_df[['departure.day_of_week']].head())


In [173]:
import pandas as pd

# Function to extract the day of the week from the scheduled time
def extract_day_of_week_from_time(row):
    # Use 'departure.scheduledTime' if the 'departure.day_of_week' is -1
    if row['departure.day_of_week'] == -1:
        return row['departure.scheduledTime'].weekday()  # Returns an integer (0 = Monday, 6 = Sunday)
    return row['departure.day_of_week']

# Apply the function to both train and test dataframes
train_df['departure.day_of_week'] = train_df.apply(extract_day_of_week_from_time, axis=1)
test_df['departure.day_of_week'] = test_df.apply(extract_day_of_week_from_time, axis=1)

# Verify the result to see if -1 values have been fixed
print(train_df['departure.day_of_week'].value_counts())
print(test_df['departure.day_of_week'].value_counts())


departure.day_of_week
6    7612
4    7523
3    7491
5    7446
1    7335
2    7260
0    7204
Name: count, dtype: int64
departure.day_of_week
3    2244
4    2194
1    2164
6    2149
0    2101
5    2085
2    2045
Name: count, dtype: int64


In [192]:
print(train_df.info(50))
print(test_df.info(50))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51871 entries, 0 to 51870
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   status                     51871 non-null  int64         
 1   departure.iataCode         51871 non-null  object        
 2   departure.icaoCode         51871 non-null  object        
 3   departure.terminal         51871 non-null  object        
 4   departure.scheduledTime    51871 non-null  datetime64[ns]
 5   departure.estimatedTime    51871 non-null  object        
 6   departure.actualTime       51871 non-null  object        
 7   departure.estimatedRunway  51871 non-null  object        
 8   departure.actualRunway     51871 non-null  object        
 9   arrival.scheduledTime      51871 non-null  datetime64[ns]
 10  arrival.estimatedTime      51871 non-null  object        
 11  airline.iataCode           51871 non-null  object        
 12  airl

In [190]:
# Save the cleaned train and test dataframes to CSV files
train_df.to_csv('merged_features_drop_cleaned_train_data.csv', index=False)
test_df.to_csv('merged_features_drop_cleaned_test_data.csv', index=False)

# Optionally, print confirmation
print("Data saved successfully!")


Data saved successfully!


In [194]:
train_df['binary_delay'] = train_df['departure.delay_minutes'].apply(lambda x: 'on-time' if x == 0 else 'delayed')


In [196]:
from sklearn.model_selection import train_test_split

X = train_df.drop(['departure.delay_minutes', 'binary_delay'], axis=1)
y = train_df['binary_delay']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [202]:
X.dtypes

status                                int64
departure.iataCode                   object
departure.icaoCode                   object
departure.terminal                   object
departure.scheduledTime      datetime64[ns]
departure.estimatedTime              object
departure.actualTime                 object
departure.estimatedRunway            object
departure.actualRunway               object
arrival.scheduledTime        datetime64[ns]
arrival.estimatedTime                object
airline.iataCode                     object
airline.icaoCode                     object
flight.number                         int64
flight.iataNumber                    object
flight.icaoNumber                    object
arrival.actualTime                   object
arrival.estimatedRunway              object
arrival.actualRunway                 object
departure.hour_of_day                 int64
departure.month                       int64
status_encoded                        int64
Departure_Date                  